# 📊 Vendor Performance Analytics Agent

## Google AI Intensive 5-Day Capstone Project

**An intelligent agent that analyzes vendor performance using natural language queries**

---

### 🎯 What You'll Learn in This Notebook

1. **Project Overview** - What the agent does and why
2. **Architecture** - How the ADK pattern is implemented
3. **5 Analytics Tools** - Detailed explanation of each tool
4. **Memory System** - How context retention works
5. **Live Demonstrations** - 5 queries with full outputs
6. **Future Roadmap** - What's next for this project

---

### ✨ Key Features

- ✅ **100% Mock Mode** - No API key or MongoDB required
- ✅ **Natural Language** - Understands 40+ query patterns
- ✅ **Memory-Aware** - Remembers context for follow-ups
- ✅ **Smart Validation** - Helpful error messages
- ✅ **Dashboard Output** - Professional formatting
- ✅ **Decision Transparency** - Shows reasoning process

---

**Author**: Priyanshu  
**Program**: Google AI Intensive 5-Day Capstone  
**Date**: November 2025

---

## 📖 1. Introduction

### The Problem

Recruitment teams need to analyze vendor performance to make data-driven decisions:
- Which vendors provide the best candidates?
- Why are candidates getting rejected?
- How is performance trending over time?

**Challenge**: Team members lack technical skills to write database queries or navigate complex dashboards.

### The Solution

**Vendor Performance Analytics Agent** - An intelligent system that accepts natural language questions and returns actionable insights.

**Example**:
- ❌ Before: Write complex MongoDB aggregation pipeline
- ✅ After: Ask "Show me top 5 vendors in 2024"

### Why This Matters

- **Accessibility**: Anyone can analyze data, no technical skills needed
- **Speed**: Get insights in seconds, not hours
- **Accuracy**: Predefined tools ensure consistent, reliable results
- **Conversational**: Follow-up questions feel natural

---

In [ ]:
---

## 🏗️ 2. Architecture & ADK Implementation

### System Architecture (ASCII Diagram)

```
┌─────────────────────────────────────────────────────────────┐
│                    USER INPUT (Natural Language)            │
│              "Show me top 5 vendors in 2024"                │
└────────────────────────┬────────────────────────────────────┘
                         │
                         ▼
┌─────────────────────────────────────────────────────────────┐
│                   VENDOR AGENT (agent.py)                   │
│                                                             │
│  ┌──────────────────────────────────────────────────────┐  │
│  │ Step 1: NL Parser (prompts.py)                       │  │
│  │  • Pattern matching (40+ patterns)                   │  │
│  │  • Confidence scoring                                │  │
│  │  • Tool selection                                    │  │
│  │  Output: {tool: "vendor_top_performers", params: {}} │  │
│  └──────────────────────┬───────────────────────────────┘  │
│                         │                                   │
│                         ▼                                   │
│  ┌──────────────────────────────────────────────────────┐  │
│  │ Step 2: Memory Check (utils.py)                      │  │
│  │  • Check stored context                              │  │
│  │  • Auto-fill missing parameters                      │  │
│  │  • Track what was used from memory                   │  │
│  └──────────────────────┬───────────────────────────────┘  │
│                         │                                   │
│                         ▼                                   │
│  ┌──────────────────────────────────────────────────────┐  │
│  │ Step 3: Validation (validators.py)                   │  │
│  │  • Validate vendorId format                          │  │
│  │  • Validate date range (≤1 year)                     │  │
│  │  • Validate limit (1-100)                            │  │
│  │  • Return helpful errors if invalid                  │  │
│  └──────────────────────┬───────────────────────────────┘  │
│                         │                                   │
│                         ▼                                   │
│  ┌──────────────────────────────────────────────────────┐  │
│  │ Step 4: Tool Execution (tools_vendor.py)             │  │
│  │  • Execute predefined MongoDB aggregation            │  │
│  │  • OR return mock data (if mock mode)                │  │
│  │  • Return structured result                          │  │
│  └──────────────────────┬───────────────────────────────┘  │
│                         │                                   │
│                         ▼                                   │
│  ┌──────────────────────────────────────────────────────┐  │
│  │ Step 5: Output Formatting (utils.py)                 │  │
│  │  • Dashboard-quality formatting                      │  │
│  │  • Visual indicators (🟢🟡🔴)                         │  │
│  │  • Progress bars, medals, trend arrows               │  │
│  └──────────────────────┬───────────────────────────────┘  │
└─────────────────────────┼───────────────────────────────────┘
                          │
                          ▼
┌─────────────────────────────────────────────────────────────┐
│                   FORMATTED OUTPUT                          │
│         🏆 TOP PERFORMING VENDORS LEADERBOARD               │
│  🥇 #1 | VENDOR_1 | 18 onboarded | 40.0% ████░░░░░░        │
│  🥈 #2 | VENDOR_3 | 16 onboarded | 38.0% ███░░░░░░░        │
│  🥉 #3 | VENDOR_2 | 15 onboarded | 39.0% ███░░░░░░░        │
└─────────────────────────────────────────────────────────────┘
```

### ADK Concepts Demonstrated

| ADK Concept | Our Implementation |
|-------------|-------------------|
| **Tool-Calling** | 5 predefined tools, pattern-based selection |
| **Short-Term Memory** | Stores vendorId, dateRange, lastNWeeks |
| **Multi-Step Planning** | Parse → Memory → Validate → Execute → Format |
| **Error Handling** | Structured errors with suggestions |
| **Decision Transparency** | Shows tool selection reasoning |

---

In [ ]:
---

## 🛠️ 3. The 5 Analytics Tools

Each tool is a predefined MongoDB aggregation pipeline. No LLM generates queries - ensuring reliability and security.

### Tool 1: `get_vendor_summary`
**Purpose**: Get performance metrics for a single vendor

**Parameters**:
- `vendorId` (string): Vendor identifier (e.g., "VENDOR_1")
- `dateRange` (dict): Start and end dates

**MongoDB Aggregation**:
```javascript
[
  { $match: { vendorId: "VENDOR_1", date: { $gte: start, $lte: end } } },
  { $group: {
      _id: "$status",
      count: { $sum: 1 },
      avgDays: { $avg: "$daysToOnboard" }
  }},
  { $project: { /* calculate join ratio, format output */ }}
]
```

**Output**: Shared count, interviewed count, onboarded count, join ratio, avg time to hire

---

### Tool 2: `compare_vendors`
**Purpose**: Side-by-side comparison of two vendors

**Parameters**:
- `vendorA` (string): First vendor ID
- `vendorB` (string): Second vendor ID  
- `dateRange` (dict): Date range for comparison

**Logic**: Runs two parallel aggregations (one per vendor) and combines results

**Output**: Metrics for both vendors with winner indication

---

### Tool 3: `get_vendor_trend`
**Purpose**: Weekly performance trends over time

**Parameters**:
- `vendorId` (string): Vendor to analyze
- `lastNWeeks` (int): Number of weeks (1-52)

**MongoDB Aggregation**:
```javascript
[
  { $match: { vendorId: "VENDOR_1", date: { $gte: startDate } } },
  { $group: {
      _id: { week: { $week: "$date" }, year: { $year: "$date" } },
      shared: { $sum: 1 },
      interviewed: { $sum: { $cond: [{ $eq: ["$status", "interviewed"] }, 1, 0] }},
      onboarded: { $sum: { $cond: [{ $eq: ["$status", "onboarded"] }, 1, 0] }}
  }},
  { $sort: { "_id.year": 1, "_id.week": 1 } }
]
```

**Output**: Array of weekly data with trend indicators

---

### Tool 4: `vendor_top_performers`
**Purpose**: Leaderboard of best vendors by join ratio

**Parameters**:
- `limit` (int): Number of results (1-100)
- `dateRange` (dict): Date range for analysis

**MongoDB Aggregation**:
```javascript
[
  { $match: { date: { $gte: start, $lte: end } } },
  { $group: {
      _id: "$vendorId",
      shared: { $sum: 1 },
      onboarded: { $sum: { $cond: [{ $eq: ["$status", "onboarded"] }, 1, 0] }}
  }},
  { $project: {
      vendorId: "$_id",
      shared: 1,
      onboarded: 1,
      joinRatio: { $divide: ["$onboarded", "$shared"] }
  }},
  { $sort: { joinRatio: -1 } },
  { $limit: limit }
]
```

**Output**: Sorted list with medals for top 3

---

### Tool 5: `vendor_failed_submissions`
**Purpose**: Analyze why candidates are rejected

**Parameters**:
- `dateRange` (dict): Date range for analysis

**MongoDB Aggregation**:
```javascript
[
  { $match: {
      status: "rejected",
      date: { $gte: start, $lte: end }
  }},
  { $group: {
      _id: "$rejectionReason",
      count: { $sum: 1 }
  }},
  { $sort: { count: -1 } },
  { $limit: 10 }
]
```

**Output**: Total rejections, top reasons with percentages, actionable insights

---

---

## 🧠 4. How Memory Works

### The Memory System

The agent maintains **short-term memory** to enable conversational interactions:

```python
# Memory stores 3 key values:
{
  "vendorId": "VENDOR_1",        # Last vendor queried
  "dateRange": {...},             # Last date range used
  "lastNWeeks": 8                 # Last week count specified
}
```

### Memory Flow

1. **First Query**: User asks "show vendor summary for VENDOR_1 in 2024"
   - Agent extracts: vendorId=VENDOR_1, dateRange={...}
   - Stores in memory

2. **Follow-up Query**: User asks "now show trend"
   - Query is vague (missing vendorId and time period)
   - Agent checks memory
   - Finds vendorId=VENDOR_1 from previous query
   - Auto-fills the missing parameter
   - Executes with complete parameters

### Why This Matters

- **Natural Conversation**: Users don't need to repeat context
- **Reduced Typing**: Follow-ups are shorter and faster
- **Better UX**: Feels like talking to a human analyst

### Implementation

Memory is checked during the **validation step**:

```python
# validators.py
def validate_vendor_id(vendor_id, memory=None):
    if not vendor_id and memory:
        vendor_id = memory.get_last_vendor_id()  # Get from memory
        if vendor_id:
            from_memory = True
    # ... validation continues
```

---

In [ ]:
---

## 🚀 5. Live Demonstrations

Let's see the agent in action! We'll run 5 queries demonstrating all features.

### Setup

### 💡 What Happened?

The agent:
1. **Parsed** the natural language query
2. **Selected** the `get_vendor_summary` tool
3. **Extracted** parameters: vendorId and dateRange
4. **Validated** all inputs
5. **Executed** the tool with mock data
6. **Formatted** the output as a professional dashboard

---

## 🔄 Query 2: Vendor Comparison

Now let's compare two vendors side-by-side.

In [ ]:
query2 = "compare VENDOR_1 and VENDOR_2 in 2024"
response2 = agent.process_query(query2)

print(f"📝 Query: {query2}")
print(agent.get_decision_explanation(response2))
print(response2['formatted'])

### 💡 Key Features

- **Side-by-side metrics** for easy comparison
- **Winner indication** with 🏆 trophy icons
- **Automatic winner determination** based on join ratio

---

## 🏆 Query 3: Top Performers

Let's see who the top performing vendors are.

In [ ]:
query3 = "top 5 vendors in 2024"
response3 = agent.process_query(query3)

print(f"📝 Query: {query3}")
print(agent.get_decision_explanation(response3))
print(response3['formatted'])

### 💡 Leaderboard Features

- **Medal system** (🥇🥈🥉) for top 3
- **Visual performance bars** showing join ratio
- **Complete metrics** for each vendor

---

---

## 🎯 Summary

### Features Demonstrated:

✅ **Mock Mode** - Works 100% offline, no API key or database needed  
✅ **Natural Language** - Understands various query patterns  
✅ **Smart Validation** - Helpful error messages with suggestions  
✅ **Memory System** - Remembers context for follow-up queries  
✅ **Decision Transparency** - Shows how queries are interpreted  
✅ **Dashboard Output** - Professional, visual formatting  
✅ **5 Analytics Tools** - Summary, comparison, trend, top performers, rejections  

### Example Queries You Can Try:

```python
"show vendor summary for VENDOR_1"
"compare VENDOR_1 and VENDOR_2"
"top 3 vendors last month"
"why are candidates rejected"
"show trend for VENDOR_2 last 8 weeks"
"now show me trend"  # Follow-up using memory
```

---

## 🚀 Future Enhancements

Planned improvements include:
- Real Google Gemini integration
- Web UI with visualizations
- Export capabilities
- Advanced analytics

---

## 👤 Author

**Priyanshu**  
Google AI Intensive 5-Day Capstone Project  
Vendor Analytics Agent with Memory & Validation

---

In [ ]:
query4 = "why are candidates rejected in 2024"
response4 = agent.process_query(query4)

print(f"📝 Query: {query4}")
print(agent.get_decision_explanation(response4))
print(response4['formatted'])

### 💡 Actionable Insights

- **Visual breakdown** of rejection reasons
- **Percentage distribution** with progress bars
- **Actionable recommendations** for improvement

---

## 🧠 Memory & Follow-up Queries

**Phase-2 Feature:** The agent remembers context from previous queries!

Let's demonstrate memory continuity with a follow-up query.

In [ ]:
# First query: Establish context
query_initial = "show vendor summary for VENDOR_3 in 2024"
response_initial = agent.process_query(query_initial)

print("=" * 60)
print("INITIAL QUERY")
print("=" * 60)
print(f"📝 Query: {query_initial}")
print(agent.get_decision_explanation(response_initial))
print(response_initial['formatted'])

In [ ]:
# Follow-up query: Uses memory for vendorId
query_followup = "now show trend for last 8 weeks"
response_followup = agent.process_query(query_followup)

print("\n" + "=" * 60)
print("FOLLOW-UP QUERY (Using Memory)")
print("=" * 60)
print(f"📝 Query: {query_followup}")
print(agent.get_decision_explanation(response_followup))

# Show which memory fields were used
if response_followup['memory_used']:
    print(f"\n🧠 Memory Used: {', '.join(response_followup['memory_used'])}")
    print(f"   → Agent remembered VENDOR_3 from the previous query!")

print(response_followup['formatted'])

### 💡 How Memory Works

The agent stores:
- **vendorId** - Last vendor queried
- **dateRange** - Last date range used
- **lastNWeeks** - Last week count specified

When you ask a vague follow-up like "now show trend", the agent automatically fills in missing parameters from memory!

---

## 🔍 Checking Memory State

Let's peek at what the agent remembers.

In [ ]:
memory_state = agent.memory.get_memory_summary()

print("🧠 Current Memory State:")
print(json.dumps(memory_state, indent=2))

---

## 🎯 Summary

### Phase-2 Features Demonstrated:

✅ **Mock Mode** - Works 100% offline, no API key or database needed  
✅ **Natural Language** - Understands various query patterns  
✅ **Smart Validation** - Helpful error messages with suggestions  
✅ **Memory System** - Remembers context for follow-up queries  
✅ **Decision Transparency** - Shows how queries are interpreted  
✅ **Dashboard Output** - Professional, visual formatting  
✅ **5 Analytics Tools** - Summary, comparison, trend, top performers, rejections  

### Example Queries You Can Try:

```python
"show vendor summary for VENDOR_1"
"compare VENDOR_1 and VENDOR_2"
"top 3 vendors last month"
"why are candidates rejected"
"show trend for VENDOR_2 last 8 weeks"
"now show me trend"  # Follow-up using memory
```

---

## 🚀 Next Steps (Phase-3)

Future enhancements will include:
- Real Google Gemini integration
- Web UI with visualizations
- Export capabilities
- Advanced analytics

---

## 👤 Author

**Priyanshu**  
Google AI Intensive 5-Day Capstone Project  
Phase-2: Enhanced Mock Mode with Memory & Validation

---

In [ ]:
# Cleanup
agent.close()
print("✅ Agent closed successfully")
print("\n🎉 Demo complete! Thank you for exploring the Vendor Analytics Agent.")